In [257]:
import numpy as np
import pandas as pd


#### some function

In [258]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))


def relu(Z):
    return np.maximum(0, Z)


def sigmoid_back(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)


def relu_backward(dA, Z):
    # print(dA.shape)
    # print("z", Z.shape)
    # print(Z <= 0)
    dZ = np.array(dA, copy=True)
    # print(dZ)
    dZ[Z <= 0] = 0

    # print("1")
    return dZ


#### DATA process

In [259]:
train_df = pd.read_csv("3.csv")
train_df = train_df.drop(['PassengerId'], axis=1)
print(train_df.head)
idx = np.random.permutation(train_df.shape[0])
# print(idx)
train_df = train_df.iloc[idx]
test_df, train_df = train_df[0: 200], train_df[200:]


<bound method NDFrame.head of      Survived  Pclass  Sex       Age  SibSp     Parch      Fare  Embarked
0         0.0     1.0  0.0  0.271174  0.125  0.000000  0.014151       0.0
1         1.0     0.0  1.0  0.472229  0.125  0.000000  0.139136       0.5
2         1.0     1.0  1.0  0.321438  0.000  0.000000  0.015469       0.0
3         1.0     0.0  1.0  0.434531  0.125  0.000000  0.103644       0.0
4         0.0     1.0  0.0  0.434531  0.000  0.000000  0.015713       0.0
..        ...     ...  ...       ...    ...       ...       ...       ...
886       0.0     0.5  0.0  0.334004  0.000  0.000000  0.025374       0.0
887       1.0     0.0  1.0  0.233476  0.000  0.000000  0.058556       0.0
888       0.0     1.0  1.0  0.336517  0.125  0.333333  0.045771       0.0
889       1.0     0.0  0.0  0.321438  0.000  0.000000  0.058556       0.5
890       0.0     1.0  0.0  0.396833  0.000  0.000000  0.015127       1.0

[891 rows x 8 columns]>


#### model

In [260]:
class model():
    def __init__(self, input_dim, learning_rate):
        super(model, self).__init__()
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.size_l = [(input_dim, 8), (8, 4), (4, 2), (2, 1)]
        self.deep = len(self.size_l)
        w = [np.ndarray(shape=(1, 1), dtype=float) for i in range(self.deep+1)]
        # self.da = self.db = self.dw = self.dz = self.a = self.z = self.b = self.w = w
        
        self.da = w.copy()
        self.w = w.copy()
        self.db = w.copy()
        self.dw = w.copy()
        self.dz = w.copy()
        self.a = w.copy()
        self.z = w.copy()
        self.b = w.copy()

        self.mw = w.copy()
        self.vw = w.copy()
        self.mb = w.copy()
        self.vb = w.copy()



        self.C = 0.001
        self.alpha = 0.8
        self.belta1 = 0.999
        self.belta2 = 0.999
        self.ep = 0.5

        
        # 初始化w,b
        
        for idx in range(len(self.size_l)):
            self.w[idx + 1] = np.random.randn(
                self.size_l[idx][1], self.size_l[idx][0]) * 0.1
            print(idx+1)
            print("---------------------------")
            self.mw[idx + 1] = np.zeros((
                self.size_l[idx][1], self.size_l[idx][0]))
            self.vw[idx + 1] = np.zeros((
                self.size_l[idx][1], self.size_l[idx][0]))
            # print("xx", self.size_l[idx][1], self.size_l[idx][0])
            # print(self.w[idx+1].shape)
            self.b[idx + 1] = np.random.randn(self.size_l[idx][1], 1) * 0.1
            self.mb[idx + 1] = np.zeros((self.size_l[idx][1], 1))
            self.vb[idx + 1] = np.zeros((self.size_l[idx][1], 1))
        # for idx in range(len(self.size_l)):
            # print(self.w[idx+1].shape)


    def linear(self, m_pre, m, ind):
        a_pre = self.a[ind - 1]
        w = self.w[ind]
        b = self.b[ind]
        z = np.dot(w, a_pre) + b
        return z

    def forward(self, x):
        # x = self.layers(x)
        self.a[0] = x
        for i in range(len(self.size_l)):
            m_pre = self.size_l[i][0]
            m = self.size_l[i][1]
            self.z[i+1] = self.linear(m_pre, m, i + 1)
            if i == len(self.size_l) - 1:
                self.a[i+1] = sigmoid(self.z[i+1])
            else:
                self.a[i+1] = relu(self.z[i+1])

                # U = (np.random.rand(*self.a[i+1].shape) < self.p)
                # self.a[i+1] *= U
                # self.a[i+1] += 1e-10

            

            # print(self.z[i+1].shape)
        return

    def backward(self, Y, s):
        L = len(self.size_l)
        a, z, w = self.a, self.z, self.w
        self.da[L] = -np.divide(Y, a[L][0]) + np.divide(1 - Y, 1 - a[L][0])
        if s == 'relu':
            back_activate = relu_backward
        else:
            back_activate = sigmoid_back

        
        for i in reversed(range(1, L + 1)):
            if i == L:
                self.dz[i] = back_activate(self.da[i], z[i])
            else:
                self.dz[i] = relu_backward(self.da[i], z[i])
            self.dw[i] = np.dot(self.dz[i], a[i-1].T)
            self.db[i] = np.sum(self.dz[i])
            self.da[i-1] = np.dot(w[i].T, self.dz[i])

    def update_adam (self, w_grad, t, m1, v1):
        m = m1 * self.belta1 + w_grad * (1 - self.belta1)
        v = v1 * self.belta2 + (w_grad * w_grad) * (1 - self.belta2)
        # t += 1
        
        m2 = m / (1 - (self.belta1 ** t))
        v2 = v / (1 - (self.belta2 ** t))
        lp = self.alpha * m2 / (np.sqrt(v2) + self.ep)  
        return lp, m, v     

    def update_argument(self, tim):
        L = len(self.size_l)
        for idx in range(1, L + 1):
            ddw, self.mw[idx], self.vw[idx] = self.update_adam(self.dw[idx], tim, self.mw[idx], self.vw[idx])
            ddb, self.mb[idx], self.vb[idx] = self.update_adam(self.db[idx], tim, self.mb[idx], self.vb[idx])
            self.w[idx] -= self.learning_rate * ddw
            self.b[idx] -= self.learning_rate * ddb
            
    def cal_loss(self, y):
        y_pred = self.a[self.deep]
        # print(y_pred)
        loss = (-1.0 / y.shape[0]) * np.sum(y * np.log(y_pred + 1e-10) + (
            1 - y) * np.log(1 - y_pred + 1e-10))
        sum = 0
        # print(y_pred[0][1])
        # print(y.shape)
        # print(y_pred)
        for i in range(y.shape[0]):
            if abs(y[i] - y_pred[0][i]) <= 0.5:
                sum += 1
        print("acc: ", sum / y.shape[0])
        print("-------------", loss)

    def train(self, x, Y, tim):
        self.forward(x)
        self.cal_loss(Y)
        self.backward(Y, 'sigmoid')
        self.update_argument(tim + 1)


#### some arugement

In [261]:
epoch = 10000
Learning_rate = 1e-3
Input_dim = 7
# np.random.seed(10)


In [262]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
604,1.0,0.0,0.0,0.434531,0.0,0.0,0.051822,0.5
502,0.0,1.0,1.0,0.316411,0.0,0.0,0.014891,1.0
425,0.0,1.0,0.0,0.349272,0.0,0.0,0.014151,0.0
463,0.0,0.5,0.0,0.597889,0.0,0.0,0.025374,0.0
619,0.0,0.5,0.0,0.321438,0.0,0.0,0.020495,0.0


#### train

In [263]:

lis = [i for i in range(train_df.shape[0])]
print(lis)
np.random.shuffle(lis)
print(lis)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [264]:
from random import shuffle

my_model = model(input_dim = Input_dim, learning_rate = Learning_rate)
for idx in range(epoch):
    # 挑选一些数据
    

    lis = [i for i in range(train_df.shape[0])]
    # print(lis)
    shuffle(lis)
    # print(lis)

    train_y = train_df['Survived']
    train_x = train_df.drop(['Survived'], axis=1)
    tr_x = train_x.to_numpy()
    tr_y = train_y.to_numpy()    

    # train = train_df[lis[:200]]
    tr_x = tr_x[lis[:200]]
    tr_y = tr_y[lis[:200]]


    # print(tr_x.shape)
    # print(tr_y.shape)
    # for j in range(tr_x.shape[0]):
    model.train(my_model, tr_x.T, tr_y, idx)
    


1
---------------------------
2
---------------------------
3
---------------------------
4
---------------------------
acc:  0.6
------------- 0.6886467110640844
acc:  0.64
------------- 0.6866191329937203
acc:  0.64
------------- 0.6865056393366267
acc:  0.6
------------- 0.688415016240975
acc:  0.61
------------- 0.6878247052679808
acc:  0.59
------------- 0.6887858641366174
acc:  0.6
------------- 0.6881844536565458
acc:  0.64
------------- 0.685946446265234
acc:  0.63
------------- 0.6863843783104835
acc:  0.6
------------- 0.6879560309025995
acc:  0.58
------------- 0.6890149331148208
acc:  0.615
------------- 0.6869392754188158
acc:  0.555
------------- 0.6903634224835616
acc:  0.655
------------- 0.6843892707864828
acc:  0.62
------------- 0.6863759946775556
acc:  0.65
------------- 0.6844488824556564
acc:  0.6
------------- 0.687432955850725
acc:  0.605
------------- 0.68704483999389
acc:  0.655
------------- 0.6837774156103329
acc:  0.625
------------- 0.6855949717764274
acc: